<a href="https://colab.research.google.com/github/harshitadd/CrossPriv/blob/master/PreVID_PySyft_Federated_Averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install syft==0.2.6 --quiet 
# !pip install pydicom --quiet

In [ ]:
import os 
from google.colab import drive
drive.mount('/content/drive')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
from torch.utils.data import TensorDataset, DataLoader
import time
import csv 
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from syft.frameworks.torch.fl import utils
from syft.workers.websocket_client import WebsocketClientWorker
import pydicom 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler 
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def makeNonFedData():
  dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch/')
  dcm_data={}
  alpha = 1.5 
  beta = 0
  labels=[]
  pid=[]
  dicom=[]
  label=[]
  for file in dcm_path:
    name = '/content/drive/My Drive/Fed_Covid/minibatch/' + file
    temp = pydicom.dcmread(name)
    image = temp.pixel_array
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    adjusted = cv2.resize(image,(128,128))
    dcm_data[file]=adjusted  
  with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
    reader = csv.reader(file)
    for row in reader:
      labels.append(row)  
  scaler = MinMaxScaler()
  cid = 0
  for PID in labels:
    for key in dcm_data:
      if(key[:-4]==PID[0]):
        l=[]
        for val in dcm_data[key]:
          l.append(scaler.fit_transform(val))
        l = np.reshape(l,(3,128,128))
        dicom.append(l)
        label.append(int(PID[5]))
  
  return dicom,label
    

In [ ]:
dicom, label = makeNonFedData()

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 8
        self.test_batch_size = 4
        self.epochs = 1
args = Arguments()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dicom,label,test_size=0.3)
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test
x_train= np.array(x_train)
y_train= np.array(y_train)
x_test= np.array(x_test)
y_test= np.array(y_test)

In [ ]:
class data_maker(Dataset):
  def __init__(self, images, labels):
        self.data = images 
        self.targets = labels 

        self.to_torchtensor()
        
  def __len__(self):
      return len(self.data)

  def to_torchtensor(self):      
      self.data=torch.from_numpy(self.data)
      self.labels=torch.from_numpy(self.targets)
  
  def __getitem__(self, idx):
      sample=self.data[idx]
      target=self.targets[idx]
      return sample,target

In [ ]:
hook = sy.TorchHook(torch)
hospital = sy.VirtualWorker(hook, id="hospital")  
clinic = sy.VirtualWorker(hook, id="clinic")  

In [ ]:
federated_train_data = data_maker(x_train,y_train).federate((hospital,clinic))

In [ ]:
federated_train_loader = sy.FederatedDataLoader(federated_train_data,batch_size=args.batch_size)
test_data = data_maker(x_test,y_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.test_batch_size)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()  
        self.conv1 = nn.Conv2d(3,32, kernel_size=8, stride = 2)  
        self.pool1 = nn.MaxPool2d(kernel_size=4,stride=2,padding=0)  
        self.conv2 = nn.Conv2d(32, 64, 8)
        self.pool2 = nn.MaxPool2d(8,8,padding=0)

        # Linear Layers 
        
        self.fc1 = nn.Linear(64*2*2, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        # print(x.shape)
        x = F.relu(self.conv1(x))
        # print(x.shape)
        x = self.pool1(x)
        # print(x.shape)
        x = F.relu(self.conv2(x))
        # print(x.shape)
        x = self.pool2(x)
        # print(x.shape)
        x = x.view(-1,64*2*2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
 
        return F.log_softmax(x,dim=1)

In [ ]:
hospital_model = Net().double()
clinic_model = Net().double()
hospital_optimizer = optim.Adam(hospital_model.parameters(), lr=0.0001)
clinic_optimizer = optim.Adam(clinic_model.parameters(), lr=0.0001)
models = [hospital_model, clinic_model]
optimizers = [hospital_optimizer, clinic_optimizer]
model = Net().double()

In [ ]:
compute_nodes = [hospital, clinic]

In [ ]:
def update(data, target, model, optimizer):
    loss = nn.CrossEntropyLoss()
    model.send(data.location) 
    optimizer.zero_grad()
    output = model(data)
    out = loss(output, target)
    out.backward()
    optimizer.step()
    return model

In [ ]:
def train():
  for batch_idx, (data, target) in enumerate(federated_train_loader):
    for index in range(len(compute_nodes)):
      models[index] = update(data,target,models[index],optimizers[index]) 
    for model in models:
      model.get()
    return utils.federated_avg({"hospital": models[0],"clinic":models[1]}) #


In [ ]:
# for i,val in enumerate(federated_train_loader):
#   print(val[0][0][0][0][0])

In [ ]:
# for i,val in enumerate(test_loader):
#   print(val[0][0][0][0][0])

In [ ]:
def test(federated_model):
  federated_model.eval()
  test_loss = 0
  correct = 0
  loss = nn.CrossEntropyLoss()
  with torch.no_grad():
    for data, target in test_loader:
          output = model(data)
          out = loss(output,target)
          test_loss += out
          pred = output.argmax(1, keepdim=True)
          correct += pred.eq(target.view_as(pred))

  test_loss /= len(test_loader.dataset)
  print(correct)
  print('For Global Model Test loss:' + str(test_loss))

In [ ]:
for epoch in range(5):
  start_time = time.time()
  print(f"Epoch Number {epoch}")
  federated_model = train()
  total_time = time.time() - start_time 
  print('Comm time', round(total_time,2),'s\n')

Epoch Number 0
Loss for local iteration (Wrapper)>[PointerTensor | me:78526597205 -> hospital:68451090450]
Loss for local iteration (Wrapper)>[PointerTensor | me:93920035238 -> hospital:94201163823]
Comm time 0.72 s

Epoch Number 1
Loss for local iteration (Wrapper)>[PointerTensor | me:12954878799 -> hospital:4110708716]
Loss for local iteration (Wrapper)>[PointerTensor | me:22337327273 -> hospital:33106492512]
Comm time 0.7 s

Epoch Number 2
Loss for local iteration (Wrapper)>[PointerTensor | me:35929974157 -> hospital:91467735734]
Loss for local iteration (Wrapper)>[PointerTensor | me:71801368172 -> hospital:4244658442]
Comm time 0.71 s

Epoch Number 3
Loss for local iteration (Wrapper)>[PointerTensor | me:17995056656 -> hospital:31687279716]
Loss for local iteration (Wrapper)>[PointerTensor | me:36327091581 -> hospital:37669575752]
Comm time 0.71 s

Epoch Number 4
Loss for local iteration (Wrapper)>[PointerTensor | me:34091768896 -> hospital:2876199771]
Loss for local iteration (Wra

In [ ]:
test(federated_model)

Test loss of the iteration on local model -  tensor(0.6586, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(1.3163, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(2.0114, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(2.7073, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(3.3291, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(3.9482, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(4.6063, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(5.2640, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(5.9226, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(6.6930, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(7.3148, dtype=torch.float64)
Test loss of the iteration on local model -  tensor(7.9383, dtype=torch.float64)
Test loss of the iteration o